In [3]:
!pip install -q -U kaggle_environments

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 13.5 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

import math


/usr/local/lib/python3.10/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-pa

Loading environment lux_ai_s3 failed: No module named 'gymnax'


/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


# Агент 1

In [6]:
%%writefile exp_agent.py
import random
# query AGENT
query_exp_agent = [0,1,2]
def exp_agent(observation, configuration):
  global query_exp_agent
  r = query_exp_agent[0]
  query_exp_agent.pop()
  query_exp_agent.append(r)
  random.shuffle(query_exp_agent)
  return r

Writing exp_agent.py


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Агент 2

In [7]:
%%writefile baby_agent.py
# baby agent
query_baby_agent = [0,1,2]
def baby_agent(observation, configuration):
  global query_baby_agent
  r = query_baby_agent[0]
  query_baby_agent.append(0)
  query_baby_agent.append(r)
  query_baby_agent = query_baby_agent[1:]
  return r

Writing baby_agent.py


# Агент 3

In [8]:
%%writefile reactionary.py
import math

import numpy as np
import pandas as pd
import random


import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

def get_score(left_move, right_move):
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

# REACTIONARY
last_react_action = None
def reactionary(observation, configuration):
    global last_react_action

    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    elif get_score(last_react_action, observation.lastOpponentAction) <= 1:
        last_react_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_react_action




Writing reactionary.py


# Агент 4

In [9]:
%%writefile c_reactionary.py
import math
import random

def get_score(left_move, right_move):
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

last_counter_action = None
def c_reactionary(observation, configuration):
    global last_counter_action
    if observation.step == 0:
        last_counter_action = random.randrange(0, configuration.signs)
    elif get_score(last_counter_action, observation.lastOpponentAction) == 1:
        last_counter_action = (last_counter_action + 2) % configuration.signs
    else:
        last_counter_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_counter_action

Writing c_reactionary.py


# Агент 5

In [10]:
%%writefile copy_opponent.py
import random
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Writing copy_opponent.py


# Агент 6

In [11]:
%%writefile easy_random.py
import random
# EASY RANDOM
def easy_random(observation, configuration):
    return random.randint(0, 2)


Writing easy_random.py


# Агент 7

In [12]:
%%writefile custom_agent.py
# CUSTOM
opp_collect = []
def custom_agent(observation, configuration):
  global opp_collect
  r = 2
  if opp_collect :
    opp_collect.append(observation.lastOpponentAction)
    if sum(opp_collect) / float(len(opp_collect)) > 1 :
      r = 2
    elif sum(opp_collect) / float(len(opp_collect)) > 0 :
      r = 1
    else :
      r = 0
  return r

Writing custom_agent.py


# Агент 8

In [13]:
%%writefile statistical.py
# Stat
action_histogram = {}


def statistical(observation, configuration):
    global action_histogram
    if observation.step == 0:
        action_histogram = {}
        return
    action = observation.lastOpponentAction
    if action not in action_histogram:
        action_histogram[action] = 0
    action_histogram[action] += 1
    mode_action = None
    mode_action_count = None
    for k, v in action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue

    return (mode_action + 1) % configuration.signs

Writing statistical.py


# Агент 9

In [14]:
%%writefile zero_agent.py
# zero AGENT
def zero_agent(observation, configuration):
  return 0

Overwriting zero_agent.py


# Агент 10

In [15]:
%%writefile one_agent.py
# one AGENT
def one_agent(observation, configuration):
  return 1

Writing one_agent.py


# Агент 11

In [16]:
%%writefile two_agent.py
# two AGENT
def zero_agent(observation, configuration):
  return 2

Writing two_agent.py


# Агент 12

In [35]:
%%writefile like_pro_agent.py

import numpy as np
import pandas as pd

from kaggle_environments import make, evaluate

# like_pro AGENT
c = 0
def like_pro_agent(observation, configuration):
  f = 0
  global c
  if c % 2 == 0 :
    f = 2
  elif c % 3 == 0 :
    f = 1
  c += 1
  return f

Overwriting like_pro_agent.py


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Организуем турнир

In [37]:
def tournir(agents) :
  grid = []
  who_played = []
  winner = ''

  # распредели по парам
  for i in range(int(len(agents) / 2)) :
    num = i * 2
    who_played.append([agents[num], agents[num + 1]])

  round = 0

  # проводим игры по принципу олимпийской системы, проигравший вылетает
  while who_played :

    if (len(who_played[round]) == 1) and (len(who_played) == 1):
      winner = who_played[round][0]
      break

    res = evaluate('rps',
             who_played[round],
             configuration={'episodeStep': 1000})
    if res[0][0] == res[0][1] : continue
    grid.append({who_played[round][0]: res[0][0],
                 who_played[round][1]: res[0][1]})
    print(f'({who_played[round][0]}', 'vs', f'{who_played[round][1]})',
          '\t\t and WIN --->', who_played[round][0] if res[0][0] > res[0][1] else who_played[round][1])
    # подводим результаты и дальше распределяем пары
    if res[0][0] > res[0][1] :
      if len(who_played[-1]) > 1 :
        who_played.append([who_played[round][0]])
      else :
        who_played[-1].append(who_played[round][0])
    if len(who_played) == 1 :
      winner = who_played[round][0] if res[0][0] > res[0][1] else who_played[round][1]
      break
    who_played = who_played[1:]


  return winner

all_agents = ['statistical.py', 'custom_agent.py',
              'easy_random.py', 'copy_opponent.py',
              'c_reactionary.py', 'reactionary.py',
              'baby_agent.py', 'exp_agent.py',
              'zero_agent.py', 'one_agent.py',
              'two_agent.py', 'like_pro_agent.py']

print('\n WINNER IS ---->', tournir(all_agents))



(statistical.py vs custom_agent.py) 		 and WIN ---> statistical.py
(easy_random.py vs copy_opponent.py) 		 and WIN ---> easy_random.py
(c_reactionary.py vs reactionary.py) 		 and WIN ---> c_reactionary.py
(baby_agent.py vs exp_agent.py) 		 and WIN ---> exp_agent.py
(zero_agent.py vs one_agent.py) 		 and WIN ---> one_agent.py
(two_agent.py vs like_pro_agent.py) 		 and WIN ---> like_pro_agent.py
(statistical.py vs easy_random.py) 		 and WIN ---> easy_random.py

 WINNER IS ----> c_reactionary.py


# Итог
в нашем импровизированном турнире побудил агент с именем **c_reactionary.py**